# Comparison star generator

## For information about your TESS target

1. Go to https://exofop.ipac.caltech.edu/tess/
2. Enter the TIC number in the TIC search box
2. Use the RA/Dec for the `ra` and `dec` variables in the cell below.
3. Use the "TESS mag" for the variable `Cmag` in the cell below.
4. Use the "depth" for the variable `depth` in the cell below.

In [1]:
import re

import requests

from astropy.coordinates import SkyCoord

from stellarphot.visualization.comparison_functions import read_file, ComparisonViewer


## Change the settings in this cell as needed

In [2]:
directory_with_images = '..'

# 👉 This could be any image from roughly the middle of the sequence.
sample_image_for_finding_stars = 'TIC_237205154.01-S001-R013-C001-rp.fit'

# 👉 You will be able to use this name later to pull out the data you want
object_of_interest = 'TIC_237205154.01'

# 👉 Get these variable from ExoFOP-TESS or Exoplanet ETD
 
ra_object = '19:34:19.97'
dec_object = '76:39:39.91'

Cmag = 13.0024 # 👉 Use TESS magnitude from ExoFOP-TESSfor this
depth = 16908.821 / 1000 # 👉 Depth should be in parts per thousand, ExoFOP gives it in ppm.

bright_mag_limit = 9
dim_mag_limit = 16

# 👉 File to save apertures in
aperture_output_file = 'aperture_locations.fits'

TESS_target = True  # change to False (capitalized) if this is NOT a TESS object

if TESS_target:
    server = "https://www.astro.louisville.edu"
    gaia_aperture_url = server + f"/cgi-bin/gaia_to_aij/upload_request.cgi?ra={ra_object}&dec={dec_object}&mag={Cmag}&depth={depth}"

    result = requests.get(gaia_aperture_url)
    links = re.search('href="(.+)"', result.text.replace('\n', ''), )

    target_file_contents = requests.get(server + links[1])
    with open('gaia_stars.radec', 'wt') as f:
        f.write(target_file_contents.text)
    targets_from_file = read_file('gaia_stars.radec')
else:
    targets_from_file = []

In [3]:
coordinate = SkyCoord(ra=ra_object, dec=dec_object, unit=("hour", "degree"))

In [4]:
comp_viewer = ComparisonViewer(sample_image_for_finding_stars,
                           directory=directory_with_images,
                           target_mag=Cmag,
                           bright_mag_limit=bright_mag_limit,
                           dim_mag_limit=dim_mag_limit,
                           targets_from_file=targets_from_file,
                           object_coordinate=coordinate
                          )
comp_viewer.box

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


### Get the table of marked stars.

In [5]:
comp_table = comp_viewer.generate_table()

### Display information on variables

In [6]:
comp_viewer.variables

OID,n_OID,Name,V,Type,l_max,max,u_max,n_max,f_min,l_min,min,u_min,n_min,l_Period,Period,u_Period,RAJ2000,DEJ2000,coords,star_id
,,,,,,mag,,,,,mag,,,,d,,deg,deg,"deg,deg",
int32,str1,str30,uint8,str30,str1,float32,str1,str6,str1,str1,float32,str1,str6,str1,float64,str3,float64,float64,SkyCoord,int64
630107,B,WISE J193202.1+764453,0,EW,,12.574,,g,(,,0.146,,g,,0.3390060000,,293.00898,76.74839,"293.00898,76.74839",23
285280,B,GSC 04584-01483,0,EW,,11.850,,R1,,,12.170,,R1,,1.0296021000,,294.20633,76.57018,"294.20633000000004,76.57018",24
1931956,,ZTF J193754.71+765357.6,0,BY,,16.794,,r,(,,0.079,,r,,0.7471696000,,294.47798,76.89936,"294.47798,76.89936",25
1530182,,ASASSN-V J193029.01+762453.3,0,EA,,14.000,,V,,,14.210,,V,,2.0027900000,,292.62087,76.41480,"292.62087,76.4148",26


### Label the stars with numbers

In [19]:
comp_viewer.show_circle()

In [15]:
comp_viewer.remove_circle()

In [16]:
comp_viewer.show_labels()

In [17]:
comp_viewer.remove_labels()

In [18]:
comp_table['plot_label'] = plot_names

NameError: name 'plot_names' is not defined

In [ ]:
comp_table.write(aperture_output_file, overwrite=True)

### Add 2.5 arcmin circle (do this after writing comparison table!)

Otherwise the comparison table will contain an extra entry for this circle.